<a href="https://colab.research.google.com/github/royn5618/Medium_Blog_Codes/blob/master/GenAI_4_NLP_Systems/OpenAI_For_Synthetic_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00


In [2]:
import openai
import pandas as pd
import numpy as np
import time

In [3]:
import os
openai.api_key = 'your-api-key'

In [4]:
prompt = """ You are generating synthetic data for a dataset containing texts and their corresponding emotion.
In this case, you are generating text samples for emotion fear. Do not use any punctuation and keep everything in lowercase.

Re-write the following text samples in order to generate synthetic data for the emotion 'fear'.

---
1. i cant walk into a shop anywhere where i do not feel uncomfortable
2. i don t feel particularly agitated
3. i pay attention it deepens into a feeling of being invaded and helpless
4. i hate it when i feel fearful for absolutely no reason
5. i do feel insecure sometimes but who doesnt
"""

In [5]:
response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                    {"role": "user", "content": prompt}
                ],
                temperature=0
            )

In [6]:
output = response["choices"][0]["message"]["content"]
print(output)

1. i am unable to enter a store without feeling a sense of dread
2. i don't feel particularly calm
3. when i focus, it intensifies into a sensation of being overwhelmed and powerless
4. i despise the moments when i feel scared without any cause
5. i sometimes feel vulnerable and it's a feeling everyone experiences


In [8]:
df = pd.read_csv('train.txt', sep=';', header=None)
df.columns = ['text', 'label']
df.head()

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [9]:
df.shape

(16000, 2)

In [10]:
emotion = 'surprise'

In [11]:
df_surprise = df[df['label'] == 'surprise']
df_surprise

,text,label
6,ive been taking or milligrams or times recomme...,surprise
32,i have seen heard and read over the past coupl...,surprise
57,i keep feeling pleasantly surprised at his sup...,surprise
64,i am now nearly finished the week detox and i ...,surprise
129,i too feel as if i am a stranger in a strange ...,surprise
...,...,...
15871,i spent a lot of time feeling a bit stunned th...,surprise
15943,i believe the most readers feel impressed by t...,surprise
15970,i have to admit im feeling pretty overwhelmed,surprise
15974,i am feeling much like the guy in the pic abov...,surprise


In [12]:
list_df = np.array_split(df_surprise, 10)

In [14]:
prompt_template = """
You are generating synthetic data for a dataset containing texts and their corresponding emotion.
In this case, you are generating text samples for emotion {emotion}. Do not use any punctuation and keep everything in lowercase.

Re-write the following text samples separated by ';' in order to generate synthetic data for the emotion {emotion}.
---
{text}
"""

modified_texts = []
for i, each_chunk in enumerate(list_df):
  print(f"Processing chunk {i}")
  _texts = "; ".join(list(each_chunk['text']))
  response = openai.ChatCompletion.create(
                  model="gpt-4",
                  messages=[
                      {"role": "user", "content": prompt_template.format(emotion=emotion, text=_texts)}
                  ],
                  temperature=0
              )
  output = response["choices"][0]["message"]["content"]
  modified_texts.extend(output.split(';'))
  time.sleep(10)

Processing chunk 0
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7
Processing chunk 8
Processing chunk 9


In [15]:
len(modified_texts)

572

In [16]:
df_new_surprise = pd.DataFrame()
df_new_surprise['text'] = modified_texts
df_new_surprise['label'] = 'surprise'

In [17]:
df_new_surprise

,text,label
0,ive been consuming more than the recommended d...,surprise
1,over the past few days ive been impressed by ...,surprise
2,his support and adaptability in new situation...,surprise
3,im almost done with my week detox and i feel ...,surprise
4,i feel like a stranger raising my son in a pl...,surprise
...,...,...
567,i spent a lot of time feeling a bit stunned t...,surprise
568,i believe most readers are impressed by the i...,surprise
569,i have to admit i am feeling quite overwhelmed,surprise
570,i am feeling much like the guy in the pic abo...,surprise


In [18]:
df = pd.concat([df, df_new_surprise])

In [19]:
df.to_csv('updated_train.csv', index=False)

In [20]:
df.tail()

,text,label
567,i spent a lot of time feeling a bit stunned t...,surprise
568,i believe most readers are impressed by the i...,surprise
569,i have to admit i am feeling quite overwhelmed,surprise
570,i am feeling much like the guy in the pic abo...,surprise
571,i need to do this that and the other for coll...,surprise


In [21]:
df.shape

(16572, 2)

In [22]:
df_new_surprise.shape

(572, 2)